# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выберите данные по странам своей группы (совместно): <br>
    3530203_80101: Spain, Portugal, France, Italy, Belgium<br>
    3530203_80102: Germany, Netherlands <br>
    3530903_80301: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_80302: Sweden, Finland, Norway, Denmark, Iceland<br>
    
2. Попытайтесь найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Найдите и объясните хотя бы 5 правил.
3. Выведите эти правила в отдельных ячейках. 
4. Подумайте, как можно было бы использовать полученные правила на практике.

In [7]:
import pandas as pd

In [8]:
data = pd.read_csv("lastfm.csv")
data.head(20)

FileNotFoundError: [Errno 2] No such file or directory: 'lastfm.csv'

In [ ]:
data.info()

In [ ]:
data.describe()

**Удалим пропущенные значения**

In [ ]:

import numpy as np

data.replace(to_replace='[unknown]',value=np.NaN, inplace=True)
data.dropna(how='any', axis=0, inplace=True)
data.isnull().sum()

In [ ]:

data_group_by_artist = data.groupby(['user', 'sex', 'country'])['artist'].apply(lambda x: '|'.join(x)).reset_index()
data_group_by_artist

In [ ]:
data_group_by_artist['country'].value_counts()

**Отфильтруем по странам - Spain, Portugal, France, Italy, Belgium**

In [ ]:
data_group_by_artist_country = data_group_by_artist[data_group_by_artist.country.isin(['Spain', 'Portugal', 'Italy', 'Belgium', 'France'])]
data_group_by_artist_country

# Dummy encoding

In [ ]:
dummy_data_by_user = data_group_by_artist_country['artist'].str.get_dummies('|')
dummy_data = pd.concat([data_group_by_artist_country, dummy_data_by_user], axis = 1)
dummy_data

In [ ]:
filter_country_data= data[data.country.isin(['Spain','Italy','Portugal','France','Belgium'])]
filter_country_data

In [ ]:
pivot_artist = filter_country_data[['user', 'artist']].pivot_table(index='user', columns='artist', aggfunc=any, fill_value=False).astype(int)
pivot_artist.sum()

# Apriori

In [ ]:
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules, fpmax

frequent_itemsets = apriori(dummy_data.drop(columns=['user', 'sex', 'country', 'artist']),
                            min_support=0.02, use_colnames=True)

frequent_itemsets

In [ ]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

In [ ]:
rules[ 
       (rules['lift'] > 1) &
       (rules['support'] > 0.04) 
     ]

**Apriori by country**

In [ ]:
pivot_country = filter_country_data[['user','country']].pivot_table(index='user', columns = 'country', aggfunc=any, fill_value=False).astype(int)
pivot_data = pd.concat([pivot_artist,pivot_country], axis=1)
apriori_data = apriori(pivot_data, min_support=0.02, use_colnames=True)
rules_data = association_rules(apriori_data, metric='confidence', min_threshold=0.6)
rules_data.sort_values('confidence', ascending=False)

# FP-growth

In [ ]:
fpgrowth_artist = fpgrowth(pivot_artist, min_support=0.005, use_colnames=True)
len(fpgrowth_artist)

In [ ]:
fpgrowth_artist_rules = association_rules(fpgrowth_artist, metric="confidence", min_threshold=0.8)
fpgrowth_artist_rules.sort_values('support', ascending=False)

In [ ]:
fpgrowth_artist_rules['lift'].describe()

In [ ]:
fpgrowth_artist_rules_lift = fpgrowth_artist_rules[fpgrowth_artist_rules['lift'] > 15]
fpgrowth_artist_rules_lift

# FP-growth by country

In [ ]:
fpgrowth_data = fpgrowth(pivot_data, min_support=0.007, use_colnames=True)
len(fpgrowth_data)

In [ ]:
fpgrowth_country_rules = association_rules(fpgrowth_data, metric="confidence", min_threshold=0.8)
fpgrowth_country_rules.sort_values('support', ascending=False)

In [ ]:
fpgrowth_country_rules_lift = fpgrowth_country_rules[fpgrowth_country_rules['lift'] > 15]
fpgrowth_country_rules_lift

# FPmax

In [ ]:
fpmax_artist = fpmax(pivot_artist, min_support=0.05, use_colnames=True)
len(fpmax_artist)

In [ ]:
fpmax_artist_rules = association_rules(fpmax_artist, metric='support', min_threshold=0.05, support_only=True)
fpmax_artist_rules.sort_values('support', ascending=False)